In [ ]:
#%save Kernel_langage.py 1

In [ ]:
"""

Class Kernel qui contient

- pour elasticsearh :
            -parametres (host et port)
            -fonction bulk (actions, index)
            -fonction creation index (index) return r (resultat)
            -fonction delete index (index)
            -count (index) renvoie le nombre d'élement

- creation de l'index regroupement (cas Lexique)

            - index (regroupement) 
            - liste_type
            - debut nom des fichiers text des regroupents du Lexique par type
                  (debut_fichier_regroupement)
            - liste des types du Lexique
            
            


"""


Nonesudo apt autoremove

In [3]:
#%save Kernel_langage.py 1


In [3]:

from elasticsearch import Elasticsearch, helpers

class Kernel () :
    def __init__ (self, host = "localhost", 
                 port = 9200 ,
                 trace = False,
                ) :
        
        self.trace = trace
        self.host = host
        self.port = port
        
        self.elastic = Elasticsearch([{"host":host , "port":port }]  )
        
        self.liste_type = ['NOM', 'AUX', 'VER', 'ADV', 'PRE', 'ADJ', 'ONO', 'CON',
                  'ART:def', 'ADJ:ind', 'PRO:ind', 'PRO:int', 'PRO:rel',
                  'ADJ:num', 'PRO:dem', 'ADJ:dem', 'PRO:per', 'ART:ind',
                  'LIA', 'PRO:pos', 'ADJ:pos', '', 'ADJ:int']
        
        self.index_regroupement = "regroupement"
        self.settings_regroupement = { "settings" : {
                                      'index.mapping.total_fields.limit':100000,
                                     }
        }
        
        self.dico_settings = {'regroupement' : self.settings_regroupement ,
                             }
                              
                              
        
        self.structure_du_lexique = ["1_ortho" , 
                                     "3_lemme" , 
                                     "4_cgram" , 
                                     "5_genre" , 
                                     "6_nombre" , 
                                     "7_freqlemfilms2" , 
                                     "8_freqlemlivres" , 
                                     "9_freqfilms2" , 
                                     "10_freqlivres" , 
                                     "11_infover" , 
                                     "12_nbhomogr" , 
                                     "13_nbhomoph" , 
                                     "14_islem" , 
                                     "15_nblettres" , 
                                     "19_voisorth" , 
                                     "21_puorth" , 
                                     "22_puphon" , 
                                     "29_cgramortho" , 
                                     "30_deflem" , 
                                     "31_defobs" , 
                                     "32_old20" , 
                                     "33_pld20" , 
                                     "35_nbmorph" , 
                                    ]
        

        
        self.debut_fichier_regroupement = "../../../data/Lexique383_group"
        
        
        
    def bulk (self,actions, index ):
        
        rr = []
        for action in actions :
            action ["_index"] = index
            rr.append(action)
            continue
        
        # make the bulk call using 'actions' and get a response"
        response = helpers.bulk(self.elastic, rr,
                                stats_only= True, 
                                request_timeout = 360,
                                raise_on_exception = False,
                               )
        
        return response
    
    def count (self, index) :
        self.elastic.indices.refresh(index)
        r = self.elastic.cat.count(index, params={"format": "json"})
        #print ('count =',r)
        return int (r [0] ['count'])
    
    def creation_index (self, index) :

        try:
            self.elastic.indices.delete (index)
        except:
            pass
        try :
            settings = self.dico_settings [index]
        except:
            settings = {}
                              
        r = self.elastic.indices.create(index, body = settings)
        if self.trace :
            print ("creation de l'index =", r, )
        return r['acknowledged']

    def delete_index (self, index) :
        try:
            self.elastic.indices.delete (index)
        except:
            return False
        return True
    
    def search_mot (self, index, mot, zone) :
        self.elastic.indices.refresh (index)
        query = {'query' : {'match' : {zone : mot}}}
        #print ("query =", query )
        res= self.elastic.search (index= index, body = query )
        #res = self.elastic.search(index=index , body={"query": {"match_all": {}}})
        #print ([hit['_source'] for hit in res ['hits'] ['hits']])
        return [hit['_source'] for hit in res ['hits'] ['hits']]
    
    def close (self,) :
        self.elastic.close()
    
      

In [4]:
# test

#from Kernel_langage import Kernel
import time

kernel = Kernel ()
print ("creation kernel = OK")

liste = kernel.liste_type
if liste == ['NOM', 'AUX', 'VER', 'ADV', 'PRE', 'ADJ', 'ONO', 'CON', 'ART:def',
             'ADJ:ind', 'PRO:ind', 'PRO:int', 'PRO:rel', 'ADJ:num', 'PRO:dem',
             'ADJ:dem', 'PRO:per', 'ART:ind', 'LIA', 'PRO:pos', 'ADJ:pos', '',
             'ADJ:int'] :
    print ( "liste_type = OK")
else :
    print ( "liste_type = KO")
    

index = kernel.index_regroupement
if index == "regroupement" :
    print ( "regroupement = OK")
else :
    print ( "regroupement = KO")
    

r = kernel.creation_index (index,)
  
if r :
    print ( "creation_index = OK")
else:
    print ( "creation_index = KO")
    
nameFile =  kernel.debut_fichier_regroupement
if nameFile == "../../../data/Lexique383_group" :
    print ("Kernel = OK")
else :
    print ("Kernel  = KO")
    
index = "test"
r = kernel.creation_index (index)
if r :
    print ('creation index OK')
else :
    print ('creation index KO #############')
    
    
actions = [ {'test' : "test1",
            "test_mot" : "test1" },
           
            {"test": "test2",
            "test_mot" : "test2 test3" },
            ]

r =kernel.bulk (actions, index)
print ("execution du bulk = OK pour 2 document", r)

mot = "test2"
zone = "test_mot"

res  = kernel.search_mot (index, mot, zone)

if len(res) == 1 :
    print ("search sur 2 docs OK")
else :
    print ("search sur 2 docs KO ##############")

#print (kernel.count (index))
print ('on demarre big bulk :')
temps_depart = time.time()
nombre = 500
nombre_iteration = 20
for k in range(0,nombre_iteration) :
    actions = []
    for i in range(0, nombre ) :
        li = ''
        for j in range(0,20 ) :
            li += str (k)+'test' + str((100*i)+j)+ ' '
            continue
        li = li [ : -1]
        doc_unitaire  = {'test' : 'test'+str(i),
                          "test_mot" : li,
                         }
        
        actions.append(doc_unitaire)
        continue
    
    r =kernel.bulk (actions, index)
    if (nombre*k)%1000 == 0 : 
        print (r, " pour nombre de document =", nombre*(k+1) )
        #print (" li =", li)
    continue
    

    
print ("execution du bulk = OK pour total de " + str(nombre_iteration*nombre) + ' documents')
print ( "duree =", time.time() -temps_depart)


nombre_voulu = (nombre_iteration*nombre) + 2

nombre = kernel.count (index)

if nombre == nombre_voulu :
    print ('count = OK')
else :
    print ("nombre =", nombre)
    print ("nombre_voulu =", nombre_voulu)
    print ('count =KO #################')
    
mot = "0test101"
zone = "test_mot"
temps_depart = time.time()
res  = kernel.search_mot(index, mot, zone)
print ("duree =", time.time() - temps_depart)
if len(res) == 1 :
    print ("search sur ", nombre, " docs OK")
else :
    print ("search sur 2 docs KO ##############")
    
    

    
r = kernel.delete_index (index)

if r :
    print ('delete_index = OK')
else:
    print('delete_index = KO #####################')
    


kernel.close()

print ('Fin TEST')







 






creation kernel = OK
liste_type = OK
regroupement = OK
creation_index = OK
Kernel = OK
creation index OK
execution du bulk = OK pour 2 document (2, 0)
search sur 2 docs OK
on demarre big bulk :
(500, 0)  pour nombre de document = 500
(500, 0)  pour nombre de document = 1500
(500, 0)  pour nombre de document = 2500
(500, 0)  pour nombre de document = 3500
(500, 0)  pour nombre de document = 4500
(500, 0)  pour nombre de document = 5500
(500, 0)  pour nombre de document = 6500
(500, 0)  pour nombre de document = 7500
(500, 0)  pour nombre de document = 8500
(500, 0)  pour nombre de document = 9500
execution du bulk = OK pour total de 10000 documents
duree = 2.156339168548584
count = OK
duree = 0.02091217041015625
search sur  10002  docs OK
delete_index = OK
Fin TEST


10002

In [4]:
from Kernel_langage import Kernel


kernel = Kernel ()

index = kernel.index_regroupement

zone = "liste_mot_lie"

kernel.search_mot(index, "avoir", zone)



[]

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    
}
res = es.index(index="test-index", id=1, body=doc)
print(res['result'])

res = es.get(index="test-index", id=1)
print(res['_source'])

es.indices.refresh(index="test-index")
index = "test-index"
res = es.search(index=index, body={"query":{"match":  {'text': 'bonsai'}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print (hit["_source"])
    #print(" %(author)s %(text)s" % hit["_source"])
    
es.indices.delete ("test-index")

In [ ]:
kernel = 